In [1]:
from dataloader import DiveFaceDataLoader
import keras
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import tensorflow as tf
import keras_vggface
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout, Flatten, Lambda, Activation, ActivityRegularization
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, models, layers, regularizers
from keras.preprocessing import image
from keras_vggface import utils
from keras_vggface.vggface import VGGFace
from tensorflow.python.keras.backend import ndim

import tensorflow as tf
from tensorflow.compat.v1 import InteractiveSession


In [2]:
#load whole dataset
demo_data = DiveFaceDataLoader().LoadData("4K_120")
#Set UP so its usable with keras ImageDataGenerator
demo_data.rename(columns={'Image':'filename'},inplace=True)


In [3]:
my_model = 'resnet50'
resnet = VGGFace(model = my_model)

#Select the last leayer as feature embedding  
last_layer = resnet.get_layer('avg_pool').output
feature_layer = Flatten(name='flatten')(last_layer)
model_vgg=Model(resnet.input, feature_layer)

#Freeze the model
model_vgg.trainable = False


In [4]:
#Create the 3 subsets from the dataset (one per race)
#white people (about 55k images keep one per identity)
white_entries = demo_data.drop(['HN','HA','MN','MA'],axis=1)
white_entries = white_entries[demo_data['HB'] != demo_data['MB']]
white_entries = white_entries.drop_duplicates("Id")
#asian people
asian_entries = demo_data.drop(['HN','HB','MN','MB'],axis=1)
asian_entries = asian_entries[demo_data['HA'] != demo_data['MA']]
#afroamerican people
afr_entries = demo_data.drop(['HA','HB','MB','MA'],axis=1)
afr_entries = afr_entries[demo_data['HN'] != demo_data['MN']]


In [15]:
balanced_classifier = keras.Sequential([
    model_vgg,
    keras.layers.Dense(3000,activation="relu"),
    keras.layers.Dense(2,activation="softmax")]
)
balanced_classifier.compile(loss='categorical_crossentropy',metrics=['acc'])

In [17]:
#Preprocessing used for the images
def preprocess(img):
    img = np.expand_dims(img, axis=0)
    return img

In [19]:
balanced_dataset

,filename,Id,H,B
112099,4K_120/HB4K_120/10115947@N02_identity_25/11378...,20000,1,0
112106,4K_120/HB4K_120/10188119@N06_identity_5/327316...,20001,1,0
112111,4K_120/HB4K_120/100224830@N05_identity_25/9503...,20002,1,0
112116,4K_120/HB4K_120/11504096@N08_identity_116/3067...,20003,1,0
112120,4K_120/HB4K_120/100021856@N08_identity_67/9466...,20004,1,0
...,...,...,...,...
92832,4K_120/MN4K_120/10617965@N07_identity_9/344366...,16057,0,1
92833,4K_120/MN4K_120/10617965@N07_identity_9/344447...,16057,0,1
92834,4K_120/MN4K_120/10617965@N07_identity_9/344366...,16057,0,1
92835,4K_120/MN4K_120/47180203@N08_identity_46/10055...,16058,0,1


In [29]:
wh_mf = white_entries.rename(columns={'HB':'H','MB':'M'})
balanced_dataset_training = wh_mf[wh_mf['H'] == 1].head(500)
balanced_dataset_training = balanced_dataset_training.append(wh_mf[wh_mf['M'] == 1].head(500))
balanced_dataset_eval = wh_mf[wh_mf['H'] == 1].tail(250)
balanced_dataset_eval= balanced_dataset_eval.append(wh_mf[wh_mf['M'] == 1].tail(250)) 
as_mf = asian_entries.rename(columns={'HA':'H','MA':'M'})
balanced_dataset_training = balanced_dataset_training.append(as_mf[as_mf['H'] == 1].head(500))
balanced_dataset_training = balanced_dataset_training.append(as_mf[as_mf['M'] == 1].head(500))
balanced_dataset_eval = balanced_dataset_eval.append(as_mf[as_mf['H'] == 1].tail(250)) 
balanced_dataset_eval= balanced_dataset_eval.append(as_mf[as_mf['M'] == 1].tail(250)) 
af_mf = afr_entries.rename(columns={'HN':'H','MN':'M'})
balanced_dataset_training = balanced_dataset_training.append(af_mf[af_mf['H'] == 1].head(500))
balanced_dataset_training = balanced_dataset_training.append(af_mf[af_mf['M'] == 1].head(500))
balanced_dataset_eval = balanced_dataset_eval.append(af_mf[af_mf['H'] == 1].tail(250)) 
balanced_dataset_eval= balanced_dataset_eval.append(af_mf[af_mf['M'] == 1].tail(250)) 


In [30]:
training_balanced = ImageDataGenerator(preprocessing_function=preprocess).flow_from_dataframe(balanced_dataset_training,directory=".",target_size=(224,224),y_col=['H','M'],class_mode='raw')

evaluation_balanced = ImageDataGenerator(preprocessing_function=preprocess).flow_from_dataframe(balanced_dataset_eval,directory=".",target_size=(224,224),y_col=['H','M'],class_mode='raw')


Found 3000 validated image filenames.
Found 1500 validated image filenames.


In [31]:
balanced_classifier.fit(training_balanced,validation_data=evaluation_balanced,
                               epochs=3)

Epoch 1/3
94/94 [==============================] - 79s 828ms/step - loss: 7.8486 - acc: 0.8639 - val_loss: 0.2593 - val_acc: 0.9767
Epoch 2/3
94/94 [==============================] - 77s 823ms/step - loss: 0.2552 - acc: 0.9691 - val_loss: 0.2498 - val_acc: 0.9780
Epoch 3/3
94/94 [==============================] - 77s 824ms/step - loss: 0.1625 - acc: 0.9829 - val_loss: 0.3170 - val_acc: 0.9807
